# **Section 1: Setup and Imports** <a id="1"></a>

In [1]:
# Install if not already installed (uncomment below in Jupyter)
# !pip install openai python-dotenv

import openai
import os
from dotenv import load_dotenv


In [2]:
# Load the OpenAI API key from .env file
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Check to confirm it's loaded (optional)
print("✅ API key loaded:", openai.api_key is not None)

✅ API key loaded: True


# **Section 2: Functions** <a id="2"></a>
## **2.1 OpenAI Functions** <a id="2.1"></a>

In [3]:
# Function to call OpenAI's ChatCompletion API with structured messages
def call_openai(messages, model="gpt-3.5-turbo", temperature=0.7):
    """
    Calls OpenAI's ChatCompletion API with structured messages.

    Parameters:
    messages (list): A list of message dictionaries, where each dictionary contains 'role' and 'content' keys.
    model (str): The model to use for the API call. Default is "gpt-3.5-turbo".
    temperature (float): The sampling temperature to use. Higher values mean the model will take more risks. Default is 0.7.

    Workflow:
    1. The function takes the input parameters and calls the OpenAI ChatCompletion API.
    2. The API returns a response containing multiple choices.
    3. The function extracts the content of the first choice from the response.

    Returns:
    str: The content of the first choice from the API response.
    """
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response['choices'][0]['message']['content']

In [4]:
# Helper to construct system + user messages for the reasoning agent
def build_review_prompt(report_text, history=[]):
    """
    Constructs system and user messages for the reasoning agent to review a consulting report.

    Parameters:
    report_text (str): The text of the consulting report to be reviewed.
    history (list): A list of previous messages (optional). Default is an empty list.

    Workflow:
    1. The function creates a system message that sets the context for the reasoning agent.
    2. It then creates a user message containing the consulting report text.
    3. The function combines the system message, the history of previous messages, and the user message into a single list.

    Returns:
    list: A list of message dictionaries, including the system message, any historical messages, and the user message.
    """
    system_msg = {
        "role": "system",
        "content": (
            "You are an experienced IT strategy consultant. "
            "You are reviewing a consulting report for completeness, clarity, risks, and alignment with best practices. "
            "Think step-by-step and identify gaps, ask clarifying questions, or suggest improvements. "
            "Your goal is to provide helpful, critical feedback using your expert knowledge."
        )
    }

    user_msg = {
        "role": "user",
        "content": f"Here is the consulting report to review:\n\n{report_text}"
    }

    return [system_msg] + history + [user_msg]


In [5]:
# Track total tokens used and estimated cost
total_tokens_used = 0
estimated_cost_usd = 0.0

# Cost per 1K tokens for GPT-3.5-turbo (adjust if using GPT-4)
COST_PER_1K_TOKENS = 0.0015

def call_openai_with_tracking(messages, model="gpt-3.5-turbo", temperature=0.7, max_tokens=500):
    """
    Calls OpenAI's ChatCompletion API with structured messages and tracks token usage and estimated cost.

    Parameters:
    messages (list): A list of message dictionaries, where each dictionary contains 'role' and 'content' keys.
    model (str): The model to use for the API call. Default is "gpt-3.5-turbo".
    temperature (float): The sampling temperature to use. Higher values mean the model will take more risks. Default is 0.7.
    max_tokens (int): The maximum number of tokens to generate in the completion. Default is 500.

    Workflow:
    1. The function takes the input parameters and calls the OpenAI ChatCompletion API.
    2. The API returns a response containing multiple choices and token usage information.
    3. The function extracts the content of the first choice from the response.
    4. It updates the total tokens used and the estimated cost in USD.
    5. It logs the prompt tokens, completion tokens, total tokens used so far, and the estimated cost.

    Returns:
    str: The content of the first choice from the API response.
    """
    global total_tokens_used, estimated_cost_usd

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )

    usage = response['usage']
    prompt_tokens = usage.get('prompt_tokens', 0)
    completion_tokens = usage.get('completion_tokens', 0)
    total = usage.get('total_tokens', prompt_tokens + completion_tokens)

    # Update tracking
    total_tokens_used += total
    estimated_cost_usd += (total / 1000) * COST_PER_1K_TOKENS

    # Logging
    print(f"🔢 Prompt: {prompt_tokens} tokens | Completion: {completion_tokens} tokens | Total so far: {total_tokens_used} tokens")
    print(f"💰 Estimated cost so far: ${estimated_cost_usd:.4f} USD")

    return response['choices'][0]['message']['content']


## **2.2 Data Preprocessing Functions** <a id="2.2"></a>

In [6]:
# Pre-process the report into sections
def split_report_into_sections(report_text):
    """
    Splits a consulting report into sections based on headers.

    Parameters:
    report_text (str): The text of the consulting report to be split into sections.

    Workflow:
    1. The function initializes an empty dictionary to store sections and sets the current section to "Header".
    2. It splits the report text into lines and iterates through each line.
    3. For each line:
       - If the line is blank, it skips to the next line.
       - If the line ends with a colon and contains 4 or fewer words, it is considered a section header.
         The current section is updated to this header (without the colon), and a new entry is created in the dictionary.
       - Otherwise, the line is added to the current section's content.
    4. After processing all lines, the function joins the content of each section into a single string.

    Returns:
    dict: A dictionary where keys are section headers and values are the corresponding section contents.
    """
    sections = {}
    lines = report_text.strip().split("\n")
    current_section = "Header"
    sections[current_section] = []

    for line in lines:
        line = line.strip()
        if not line:
            continue  # skip blank lines
        elif line.endswith(":") and len(line.split()) <= 4: # Check if it's a section header.  If yes, create a new section with the header as the key.
            current_section = line.replace(":", "").strip()
            sections[current_section] = []
        else:
            sections[current_section].append(line) # Add the line (contents below header) to the current section

    # Join section contents with newlines
    for key in sections:
        sections[key] = "\n".join(sections[key])

    return sections


## **2.3 Static Reasoning Agent Functions** <a id="2.3"></a>

In [7]:
class ITReportReviewer:
    """
    A class to review sections of an IT consulting report using OpenAI's ChatCompletion API.

    Process:
    1. Initializes the reviewer with the report sections
    2. Lets it review one section at a time using real reasoning
    3. Stores and prints each review with feedback

    Attributes:
    sections (dict): A dictionary where keys are section headers and values are the corresponding section contents.
    model (str): The model to use for the API call. Default is "gpt-3.5-turbo".
    temperature (float): The sampling temperature to use. Higher values mean the model will take more risks. Default is 0.7.
    review_history (list): A list to store the history of reviews for each section.

    Methods:
    review_section(section_name):
        Reviews a specific section of the report using OpenAI's ChatCompletion API.
    """
    
    def __init__(self, report_sections, model="gpt-3.5-turbo", temperature=0.7):
        """
        Initializes the ITReportReviewer with the given report sections, model, and temperature.

        Parameters:
        report_sections (dict): A dictionary where keys are section headers and values are the corresponding section contents.
        model (str): The model to use for the API call. Default is "gpt-3.5-turbo".
        temperature (float): The sampling temperature to use. Higher values mean the model will take more risks. Default is 0.7.
        """
        self.sections = report_sections
        self.model = model
        self.temperature = temperature
        self.review_history = []

    def review_section(self, section_name):
        """
        Reviews a specific section of the report using OpenAI's ChatCompletion API.

        Parameters:
        section_name (str): The name of the section to review.

        Workflow:
        1. Retrieves the text of the specified section from the sections dictionary.
        2. If the section is empty or missing, prints a warning message.
        3. Builds a prompt for reviewing the section using the build_review_prompt function.
        4. Calls the OpenAI ChatCompletion API with tracking using the call_openai_with_tracking function.
        5. Saves the review in the review_history attribute.
        6. Prints the review for the specified section.

        Returns:
        None
        """
        section_text = self.sections.get(section_name, "")
        if not section_text:
            print(f"⚠️ Section '{section_name}' is empty or missing.")
            return

        # Build prompt for reviewing the section
        messages = build_review_prompt(
            report_text=f"Section: {section_name}\n\n{section_text}",
            history=[]
        )

        # Get AI-generated reasoning using tracked OpenAI call
        review = call_openai_with_tracking(messages, model=self.model, temperature=self.temperature)

        # Save the review
        self.review_history.append({
            "section": section_name,
            "review": review
        })

        print(f"\n✅ Review for section '{section_name}':\n{review}\n{'-'*60}")


In [8]:
def summarize_full_review(agent):
    # Combine all reviews into a single prompt
    combined_review_text = ""
    for step in agent.review_history:
        combined_review_text += f"Section: {step['section']}\nFeedback: {step['review']}\n\n"

    # Build new prompt asking for a final report assessment
    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert IT strategy consultant reviewing an internal report assessment. "
                "Summarize the overall quality of the report based on the following section reviews. "
                "Highlight gaps, strengths, and suggest next steps to improve the report."
            )
        },
        {
            "role": "user",
            "content": combined_review_text
        }
    ]

    summary = call_openai_with_tracking(messages)
    return summary

## **2.4 ReAct Reasoning Agent Functions** <a id="2.4"></a>

In [ ]:
class ReActConsultantAgent:
    """
    A class to review sections of an IT consulting report using the ReAct (Reason + Act) framework with OpenAI's ChatCompletion API.

    Process:
    1. Initializes the agent with the section name and text.
    2. Builds a prompt for the ReAct framework.
    3. Tracks the history of thoughts, actions, and observations.

    Attributes:
    section_name (str): The name of the section to review.
    section_text (str): The text of the section to review.
    model (str): The model to use for the API call. Default is "gpt-3.5-turbo".
    temperature (float): The sampling temperature to use. Higher values mean the model will take more risks. Default is 0.7.
    history (list): A list to store the history of thoughts, actions, and observations.

    Methods:
    build_react_prompt():
        Builds a prompt for the ReAct framework based on the section text and history.
    """

    def __init__(self, section_name, section_text, model="gpt-3.5-turbo", temperature=0.7):
        """
        Initializes the ReActConsultantAgent with the given section name, section text, model, and temperature.

        Parameters:
        section_name (str): The name of the section to review.
        section_text (str): The text of the section to review.
        model (str): The model to use for the API call. Default is "gpt-3.5-turbo".
        temperature (float): The sampling temperature to use. Higher values mean the model will take more risks. Default is 0.7.
        """
        self.section_name = section_name
        self.section_text = section_text
        self.model = model
        self.temperature = temperature
        self.history = []

    def build_react_prompt(self):
        """
        Builds a prompt for the ReAct framework based on the section text and history.

        Workflow:
        1. Constructs a base prompt with the section name and text.
        2. Iterates through the history of thoughts, actions, and observations, appending them to the base prompt.
        3. Adds a final line asking for the next thought and action.

        Returns:
        list: A list containing a single dictionary with the role 'user' and the constructed prompt as content.
        """
        base_prompt = (
            f"You are an expert IT strategy consultant reviewing a report section titled '{self.section_name}'.\n"
            "You are using ReAct (Reason + Act) to think through the review.\n\n"
            "Format each response like this:\n"
            "Thought: <your reasoning>\n"
            "Action: <one of: ask_question, flag_risk, recommend_fix, summarize>\n\n"
            f"Here is the section content:\n{self.section_text}\n\n"
        )

        for step in self.history:
            base_prompt += f"Thought: {step['thought']}\n"
            base_prompt += f"Action: {step['action']}\n"
            base_prompt += f"Observation: {step['observation']}\n\n"

        base_prompt += "What is your next Thought and Action?"

        return [{"role": "user", "content": base_prompt}]

    def build_react_prompt_withTools(self):
            """
            Builds a prompt for the ReAct framework based on the section text and history.

            Workflow:
            1. Constructs a base prompt with the section name and text.
            2. Iterates through the history of thoughts, actions, and observations, appending them to the base prompt.
            3. Adds a final line asking for the next thought and action.
            For the check_guideline action, the prompt includes a placeholder for the topic.
            LLM infers topic from the section_text

            Returns:
            list: A list containing a single dictionary with the role 'user' and the constructed prompt as content.
            """
            base_prompt = (
                f"You are an expert IT strategy consultant reviewing a report section titled '{self.section_name}'.\n"
                "You are using ReAct (Reason + Act) to think through the review.\n\n"
                "Format each response like this:\n"
                "Thought: <your reasoning>\n"
                "Action: <one of: " + format_tools_list(tool_catalog) + "'\n\n"
            )

            base_prompt += format_tool_catalog_for_prompt(tool_catalog)
            base_prompt += f"Here is the section content:\n{self.section_text}\n\n"

            for step in self.history:
                base_prompt += f"Thought: {step['thought']}\n"
                base_prompt += f"Action: {step['action']}\n"
                base_prompt += f"Observation: {step['observation']}\n\n"

            base_prompt += "What is your next Thought and Action?"

            return [{"role": "user", "content": base_prompt}]


In [10]:
def run_react_loop_static(agent, max_steps=5):
    """
    Runs the ReAct (Reason + Act) loop for a specified number of steps.

    Purpose:
    This function iterates through a reasoning and action loop using the ReAct framework to review a section of an IT consulting report. It generates thoughts, actions, and observations at each step, and stores the history of these steps.

    Parameters:
    agent (ReActConsultantAgent): An instance of the ReActConsultantAgent class, initialized with the section name and text.
    max_steps (int): The maximum number of steps to run the loop. Default is 5.

    Workflow:
    1. Iterates through the loop for a maximum of `max_steps` times.
    2. In each iteration:
    - Calls `agent.build_react_prompt()` to construct the prompt for the ReAct framework.
    - Calls `call_openai_with_tracking()` to get the response from the OpenAI API.
    - Parses the response to extract the thought and action.
    - Generates an observation based on the action.
    - Stores the thought, action, and observation in the agent's history.
    - Prints the result of the current step.
    - Breaks the loop if the action is "summarize".
    3. Returns the full reasoning history.

    Returns:
    list: A list of dictionaries, where each dictionary contains the thought, action, and observation for each step.
    """
    for step_num in range(max_steps):
        messages = agent.build_react_prompt()
        response = call_openai_with_tracking(messages, model=agent.model, temperature=agent.temperature)

        # Parse response
        try:
            lines = response.strip().split("\n")
            thought = next(line.split(":", 1)[1].strip() for line in lines if line.lower().startswith("thought"))
            action = next(line.split(":", 1)[1].strip() for line in lines if line.lower().startswith("action"))
        except:
            print("⚠️ Failed to parse model response.")
            break

        # Generate observation based on action
        if action == "ask_question":
            observation = "Good question to ask the client for clarification."
        elif action == "flag_risk":
            observation = "This is a legitimate risk that should be addressed."
        elif action == "recommend_fix":
            observation = "The recommendation improves the section's clarity and compliance."
        elif action == "summarize":
            observation = "Review complete."
        else:
            observation = "Unrecognized action."

        # Store step
        agent.history.append({
            "thought": thought,
            "action": action,
            "observation": observation
        })

        # Print result of this step
        print(f"\n🔁 Step {step_num + 1}")
        print(f"🧠 Thought: {thought}")
        print(f"⚙️ Action: {action}")
        print(f"👀 Observation: {observation}")

        if action == "summarize":
            break

    # Return full reasoning history
    return agent.history


In [ ]:
def run_react_loop_check_withTool(agent, max_steps=5):
    """
    Runs the ReAct (Reason + Act) loop for a specified number of steps.

    Purpose:
    This function iterates through a reasoning and action loop using the ReAct framework to review a section of an IT consulting report. It generates thoughts, actions, and observations at each step, and stores the history of these steps.

    Parameters:
    agent (ReActConsultantAgent): An instance of the ReActConsultantAgent class, initialized with the section name and text.
    max_steps (int): The maximum number of steps to run the loop. Default is 5.

    Workflow:
    1. Iterates through the loop for a maximum of `max_steps` times.
    2. In each iteration:
       - Calls `agent.build_react_prompt()` to construct the prompt for the ReAct framework.
       - Calls `call_openai_with_tracking()` to get the response from the OpenAI API.
       - Parses the response to extract the thought and action.
       - Generates an observation based on the action.
       - Stores the thought, action, and observation in the agent's history.
       - Prints the result of the current step.
       - Breaks the loop if the action is "summarize".
    3. Returns the full reasoning history.

    Returns:
    list: A list of dictionaries, where each dictionary contains the thought, action, and observation for each step.
    """
    for step_num in range(max_steps):
        messages = agent.build_react_prompt_checkGuideline()
        response = call_openai_with_tracking(messages, model=agent.model, temperature=agent.temperature)

        # Parse response
        try:
            lines = response.strip().split("\n")
            thought = next(line.split(":", 1)[1].strip() for line in lines if line.lower().startswith("thought"))
            action = next(line.split(":", 1)[1].strip() for line in lines if line.lower().startswith("action"))
        except:
            print("⚠️ Failed to parse model response.")
            break

        # Generate observation based on action
        if action.startswith("check_guideline"): # tool #1: check_guideline
            try:
                topic = action.split("[", 1)[1].rstrip("]").strip('"')
                observation = check_guideline(topic)
            except:
                observation = "⚠️ Could not parse check_guideline action."
        elif action.startswith("keyword_match_in_section"): # tool #2: keyword_match_in_section
            try:
                term = action.split("[", 1)[1].rstrip("]").strip('"')
                observation = keyword_match_in_section(term, agent.section_text)
            except:
                observation = "⚠️ Could not parse keyword_match_in_section action."
        elif action.startswith("check_timeline_feasibility"): # tool #3: check_timeline_feasibility
            try:
                duration = action.split("[", 1)[1].rstrip("]").strip('"')
                observation = check_timeline_feasibility(duration)
            except:
                observation = "⚠️ Could not parse check_timeline_feasibility action."
        elif action.startswith("search_report"): # tool #4: search_report
            try:
                term = action.split("[", 1)[1].rstrip("]").strip('"')
                observation = search_report(term, report_sections)
            except:
                observation = "⚠️ Could not parse search_report action."
        elif action == "ask_question": # hardcoded action
            observation = "Good question to ask the client for clarification."
        elif action == "flag_risk": # hardcoded action
            observation = "This is a legitimate risk that should be addressed."
        elif action == "recommend_fix": # hardcoded action 
            observation = "The recommendation improves the section's clarity and compliance."
        elif action == "summarize": # hardcoded action; end the loop
            observation = "Review complete."
        else:
            observation = "Unrecognized action."

        # Store step
        agent.history.append({
            "thought": thought,
            "action": action,
            "observation": observation
        })

        # Print result of this step
        print(f"\n🔁 Step {step_num + 1}")
        print(f"🧠 Thought: {thought}")
        print(f"⚙️ Action: {action}")
        print(f"👀 Observation: {observation}")

        if action == "summarize":
            break

    # Return full reasoning history
    return agent.history


In [ ]:
# Define dicionary of tools available for ReActConsultantAgent to call

tool_catalog = {
    "check_guideline": {
        "description": "Look up a best practice for a given topic",
        "usage": 'check_guideline["cloud security"]',
        "version": "v1.0",
        "examples": [
            "check_guideline[\"data governance\"]",
            "check_guideline[\"migration strategy\"]"
        ]
    },
    "keyword_match_in_section": {
        "description": "Check if a keyword appears in the current section",
        "usage": 'keyword_match_in_section["encryption"]',
        "version": "v1.0",
        "examples": [
            "keyword_match_in_section[\"stakeholders\"]"
        ]
    },
    "check_timeline_feasibility": {
        "description": "Check if a project timeline is realistic for migration",
        "usage": 'check_timeline_feasibility["12 weeks"]',
        "version": "v1.0",
        "examples": [
            "check_timeline_feasibility[\"3 months\"]"
        ]
    },
    "search_report": {
        "description": "Search the entire report for a concept or term",
        "usage": 'search_report["data governance"]',
        "version": "v1.0",
        "examples": [
            "search_report[\"Zero Trust\"]"
        ]
    },
    "ask_question": {
        "description": "Ask a clarifying question about the report",
        "usage": "ask_question",
        "version": "v1.0",
        "examples": ["ask_question"]
    },
    "flag_risk": {
        "description": "Flag a gap, issue, or concern in the section",
        "usage": "flag_risk",
        "version": "v1.0",
        "examples": ["flag_risk"]
    },
    "recommend_fix": {
        "description": "Suggest a specific improvement",
        "usage": "recommend_fix",
        "version": "v1.0",
        "examples": ["recommend_fix"]
    },
    "summarize": {
        "description": "Summarize your review and end the loop",
        "usage": "summarize",
        "version": "v1.0",
        "examples": ["summarize"]
    }
}


In [ ]:
# Format the tool catalog for display in the prompt for consumption by LLM

def format_tool_catalog_for_prompt(tool_catalog):
    """
    Formats the tool catalog for display in the prompt for consumption by LLM.

    Purpose:
    This function formats the tool catalog into a human-readable string that lists each tool along with its description and usage. This formatted string can be used in prompts for language models to understand the available tools.

    Parameters:
    tool_catalog (dict): A dictionary where keys are tool names and values are dictionaries containing tool metadata, including 'description' and 'usage'.

    Workflow:
    1. Initializes a list with the header "Available tools:".
    2. Iterates through each tool in the tool_catalog dictionary.
    3. For each tool, appends its name, description, and usage to the list.
    4. Joins the list into a single string with newline characters.

    Returns:
    str: A formatted string listing all tools with their descriptions and usage.
    """
    lines = ["Available tools:\n"]
    for tool, meta in tool_catalog.items():
        lines.append(f"- {tool} (v{meta['version']}): {meta['description']}")
        lines.append(f"  Usage: {meta['usage']}")
        if meta.get("examples"):
            for ex in meta["examples"]:
                lines.append(f"  Example: {ex}")
        lines.append("")  # spacing between tools
    return "\n".join(lines)



In [ ]:
def format_tools_list(tool_catalog):
    """
    Formats the list of tools from the tool_catalog dictionary as a comma-separated string.

    Parameters:
    tool_catalog (dict): A dictionary where keys are tool names and values are dictionaries containing tool metadata.

    Returns:
    str: A comma-separated string of tool names.
    """
    tools_list = ", ".join(tool_catalog.keys())
    return tools_list

# Example usage
formatted_tools = format_tools_list(tool_catalog)
print(formatted_tools)

In [ ]:
# Tool #1: Check for best practices in a given section

# Simulated best practices reference data
best_practices = {
    "cloud security": "Follow NIST Cybersecurity Framework. Include access control, encryption at rest/in-transit, and regular audits.",
    "data governance": "Establish data stewards, quality standards, lifecycle rules, and metadata documentation.",
    "migration": "Use phased migration, sandbox testing, rollback planning, and stakeholder communication."
}

def check_guideline(topic):
    """
    Checks for best practices related to a given topic.

    Purpose:
    This function looks up best practices for a specified topic from a predefined dictionary of best practices.

    Parameters:
    topic (str): The topic for which best practices are to be checked.

    Workflow:
    1. The function converts the topic to lowercase to ensure case-insensitive matching.
    2. It looks up the topic in the `best_practices` dictionary.
    3. If a matching guideline is found, it returns the guideline.
    4. If no matching guideline is found, it returns a message indicating that no matching guideline was found.

    Returns:
    str: The best practice guideline for the specified topic, or a message indicating no matching guideline was found.
    """
    return best_practices.get(topic.lower(), "No matching guideline found.")


In [ ]:
# Tool #2: Keyword matching in a section
# This tool helps the agent check if a keyword or concept is explicitly mentioned in the section.
# This is great for validating whether the report includes key elements (e.g., "encryption", "stakeholders", "Zero Trust").  

def keyword_match_in_section(term, section_text):
    """
    Checks if a keyword or concept is explicitly mentioned in a section of the report.

    Purpose:
    This function helps validate whether the report includes key elements by checking if a specified keyword or concept is mentioned in the section text.

    Parameters:
    term (str): The keyword or concept to search for in the section.
    section_text (str): The text of the section to search within.

    Workflow:
    1. Converts the keyword and section text to lowercase to ensure case-insensitive matching.
    2. Checks if the keyword is present in the section text.
    3. Returns a message indicating whether the keyword was found or not.

    Returns:
    str: A message indicating whether the keyword was found in the section.
    """
    term_lower = term.lower()
    if term_lower in section_text.lower():
        return f"The keyword '{term}' was found in the section."
    else:
        return f"The keyword '{term}' was NOT found in the section."


In [ ]:
# Tool #3: Check feasibility of timeline for IT migration
# This tool helps the agent assess the feasibility of a timeline for an IT migration project.
# It checks if the timeline is too short, potentially feasible, or reasonable for a full migration.

def check_timeline_feasibility(duration_str):
    """
    Checks the feasibility of a timeline for an IT migration project.

    Purpose:
    This function helps assess whether a given timeline for an IT migration project is too short, potentially feasible, or reasonable.

    Parameters:
    duration_str (str): The timeline duration as a string (e.g., "6 months", "12 weeks").

    Workflow:
    1. Converts the duration string to lowercase for consistency.
    2. Parses the duration string to extract the numeric value and unit (weeks or months).
    3. Converts the duration to months if it is specified in weeks.
    4. Evaluates the duration:
       - If less than 3 months, it is likely too short.
       - If between 3 and 12 months, it is potentially feasible depending on complexity.
       - If more than 12 months, it seems reasonable.
    5. Returns a message indicating the feasibility of the timeline.

    Returns:
    str: A message indicating whether the timeline is too short, potentially feasible, or reasonable.
    """
    # Convert string like "6 months" or "12 weeks" to months
    duration_str = duration_str.lower()
    try:
        if "week" in duration_str:
            num = int(duration_str.split()[0])
            months = num / 4
        else:
            num = int(duration_str.split()[0])
            months = num

        if months < 3:
            return f"The timeline ({duration_str}) is likely too short for a full migration."
        elif 3 <= months <= 12:
            return f"The timeline ({duration_str}) is potentially feasible depending on complexity."
        else:
            return f"The timeline ({duration_str}) seems reasonable for a full IT migration."
    except:
        return "⚠️ Could not interpret timeline format. Please use 'X months' or 'X weeks'."


In [ ]:
# Tool #4: Search for a term in the entire report
# This tool helps the agent search for a specific term in the entire consulting report.
# It returns the sections where the term was found, if any.

def search_report(term, report_sections):
    """
    Searches for a specific term in the entire consulting report.

    Purpose:
    This function helps the agent search for a specific term in the entire consulting report and returns the sections where the term was found, if any.

    Parameters:
    term (str): The term to search for in the report.
    report_sections (dict): A dictionary where keys are section headers and values are the corresponding section contents.

    Workflow:
    1. Initializes an empty list `found_in` to store the sections where the term is found.
    2. Iterates through each section in the `report_sections` dictionary.
    3. For each section, checks if the term (case-insensitive) is present in the section content.
    4. If the term is found, appends the section header to the `found_in` list.
    5. After checking all sections, returns a message indicating the sections where the term was found or a message indicating that the term was not found.

    Returns:
    str: A message indicating the sections where the term was found or a message indicating that the term was not found.
    """
    found_in = []
    for section, content in report_sections.items():
        if term.lower() in content.lower():
            found_in.append(section)
    if found_in:
        return f"The term '{term}' was found in: {', '.join(found_in)}."
    else:
        return f"The term '{term}' was NOT found anywhere in the report."

# **Section 3: Load Data** <a id="3"></a>

In [13]:
# Sample IT consulting report (replace with real one later)
# You can later replace this with a real one or load from a file.
# We can add file upload later (open("report.txt").read())
sample_report = """
Client: HealthConnect Systems
Industry: Healthcare
Project: IT Modernization Assessment

Summary:
HealthConnect currently operates on-premise infrastructure for its core clinical systems. While some departments use SaaS tools, there is no centralized cloud strategy. Security policies are documented but not consistently followed. There is no formal data governance framework. Leadership has expressed interest in migrating systems to the cloud.

Key Recommendations:
1. Conduct a cloud readiness assessment.
2. Begin phased migration of CRM and EHR systems.
3. Establish a data governance committee.
4. Update security protocols to align with NIST standards.

Timeline: Estimated at 6–12 months for full migration planning and execution.
"""

# **Section 4: Pre-process Data** <a id="4"></a>

In [14]:
# Run it on the sample report
report_sections = split_report_into_sections(sample_report)

# Display for verification
for section, content in report_sections.items():
    print(f"📌 {section}:\n{content}\n{'-'*60}")


📌 Header:
Client: HealthConnect Systems
Industry: Healthcare
Project: IT Modernization Assessment
------------------------------------------------------------
📌 Summary:
HealthConnect currently operates on-premise infrastructure for its core clinical systems. While some departments use SaaS tools, there is no centralized cloud strategy. Security policies are documented but not consistently followed. There is no formal data governance framework. Leadership has expressed interest in migrating systems to the cloud.
------------------------------------------------------------
📌 Key Recommendations:
1. Conduct a cloud readiness assessment.
2. Begin phased migration of CRM and EHR systems.
3. Establish a data governance committee.
4. Update security protocols to align with NIST standards.
Timeline: Estimated at 6–12 months for full migration planning and execution.
------------------------------------------------------------


# **Section 5: Model - Basic: Single-Hop Reasoning + Static Action Loop** <a id="5"></a>

1. Iterate through sections of report
2. Send each section to ChatGPT for feedback
3. Summarize feedback 

## **5.1 Initialize Agent** <a id="5.1"></a> ##

In [15]:
# Initialize the reasoning agent
agent = ITReportReviewer(report_sections)

## **5.2 Run Agent** <a id="5.2"></a> ##

In [16]:
# Define the order in which we want to review sections
sections_to_review = list(report_sections.keys())

# Loop through each section and generate a review
for section in sections_to_review:
    agent.review_section(section)

🔢 Prompt: 97 tokens | Completion: 87 tokens | Total so far: 184 tokens
💰 Estimated cost so far: $0.0003 USD

✅ Review for section 'Header':
This header section provides a brief overview of the client, industry, and project, which is a good start. However, I would recommend including additional information such as the date of the report, the names and roles of the team members involved in the assessment, and maybe a high-level objective or scope statement for the IT modernization assessment. This will provide more context for readers and help set the stage for the rest of the report.
------------------------------------------------------------
🔢 Prompt: 138 tokens | Completion: 442 tokens | Total so far: 764 tokens
💰 Estimated cost so far: $0.0011 USD

✅ Review for section 'Summary':
Thank you for sharing the summary of the consulting report. Here are some key points to consider and areas where further information or improvement may be needed:

1. **Current State Assessment**:
   - It's

In [17]:
# Call the summarize_full_review function to get the final summary
final_summary = summarize_full_review(agent)
print("📋 Final Summary of the Report Review:\n")
print(final_summary)

🔢 Prompt: 1101 tokens | Completion: 479 tokens | Total so far: 2982 tokens
💰 Estimated cost so far: $0.0045 USD
📋 Final Summary of the Report Review:

The internal report assessment shows a commendable effort in evaluating HealthConnect's IT modernization needs. Here is an overview of the overall quality and areas for improvement:

Strengths:
- The report provides a clear overview of HealthConnect's current IT landscape and leadership's interest in cloud migration.
- Key recommendations are outlined, indicating a proactive approach to addressing IT modernization challenges.
- The report mentions risk factors and challenges, demonstrating a comprehensive analysis.

Areas for Improvement:
1. **Header Section**:
   - Enhance the header by including the report date, team member names and roles, and a high-level objective or scope statement for clarity and context.

2. **Summary**:
   - Provide detailed insights into the existing on-premise systems, SaaS tools, and security policies at Heal

# **Section 6: Model - ReAct** <a id="6"></a>

1. **Think** about each section (with ChatGPT)
2. Decide on and take an **action**
3. **Observe** the results and loop back to step 1 with new reasoning

## **6.1 Simple Agent - Predefined Actions** <a id="6.1"></a> ##

In [18]:
# Choose a section to run ReAct on
section_name = "Summary"
section_text = report_sections.get(section_name, "")

# Initialize the ReAct agent
react_agent = ReActConsultantAgent(section_name, section_text)

# Run the ReAct loop
react_review_history = run_react_loop_static(react_agent)


🔢 Prompt: 139 tokens | Completion: 61 tokens | Total so far: 3182 tokens
💰 Estimated cost so far: $0.0048 USD

🔁 Step 1
🧠 Thought: The lack of a centralized cloud strategy and inconsistent adherence to security policies pose significant risks to the organization's data and systems.
⚙️ Action: flag_risk
👀 Observation: This is a legitimate risk that should be addressed.
🔢 Prompt: 184 tokens | Completion: 52 tokens | Total so far: 3418 tokens
💰 Estimated cost so far: $0.0051 USD

🔁 Step 2
🧠 Thought: Migrating systems to the cloud without a formal data governance framework in place could lead to data management challenges and potential compliance issues.
⚙️ Action: recommend_fix
👀 Observation: The recommendation improves the section's clarity and compliance.
🔢 Prompt: 230 tokens | Completion: 47 tokens | Total so far: 3695 tokens
💰 Estimated cost so far: $0.0055 USD

🔁 Step 3
🧠 Thought: Leadership's interest in migrating systems to the cloud presents an opportunity to enhance the organizat

## **6.2 Agent 2 - Simple Tools** <a id="6.2"></a> ##

In [19]:
# Choose a section to run ReAct on
section_name = "Key Recommendations"
section_text = report_sections.get(section_name, "")

# Initialize the ReAct agent
react_agent = ReActConsultantAgent(section_name, section_text)

# Run the ReAct loop
react_review_history = run_react_loop_check_withTool(react_agent)


🔢 Prompt: 206 tokens | Completion: 31 tokens | Total so far: 4626 tokens
💰 Estimated cost so far: $0.0069 USD

🔁 Step 1
🧠 Thought: It is important to ensure that the recommendations align with industry best practices and address potential risks.
⚙️ Action: check_guideline["cloud readiness assessment"]
👀 Observation: No matching guideline found.
🔢 Prompt: 244 tokens | Completion: 110 tokens | Total so far: 4980 tokens
💰 Estimated cost so far: $0.0075 USD

🔁 Step 2
🧠 Thought: Since there was no specific guideline found for cloud readiness assessment, it may be beneficial to further research industry best practices in this area to ensure the recommendation is comprehensive and effective.
⚙️ Action: recommend_fix
👀 Observation: The recommendation improves the section's clarity and compliance.
🔢 Prompt: 298 tokens | Completion: 28 tokens | Total so far: 5306 tokens
💰 Estimated cost so far: $0.0080 USD

🔁 Step 3
🧠 Thought: The phased migration of CRM and EHR systems could introduce potential